In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

api_key = 'RGAPI-887a80ea-37e1-449c-858c-f3dd59dc96e6'
watcher = LolWatcher(api_key)
my_region = 'na1'

me = watcher.summoner.by_name(my_region, 'Desuuuuuuuuuu')

latest = watcher.data_dragon.versions_for_region(my_region)['n']['champion']
static_champ_list = watcher.data_dragon.champions(latest, False, 'en_US')
my_matches = watcher.match.matchlist_by_account(my_region, me['accountId'])

In [8]:
def matchHistory(num):
    #==== just getting the match details, ranges from 0th - 99th match hence "num" ====#
    last_match = my_matches['matches'][num]
    match_detail = watcher.match.by_id(my_region, last_match['gameId'])
    #==== just getting the match details, ranges from 0th - 99th match hence "num" ====#
    
    participantsID = []
    
    #==== match_detail has to be looked through in a for loop because it is a list ====#
    for r_name in match_detail['participantIdentities']:
    #==== match_detail has to be looked through in a for loop because it is a list ====#
    
        participants = []
        for row in match_detail['participants']:
            #==== Im doing this so sift out my play info only, not the entire teams ====#
            if(r_name['player']['summonerName'] == "Desuuuuuuuuuu"):
            #==== Im doing this so sift out my play info only, not the entire teams ====#
                p_name = {}
                p_name['name'] = r_name['player']['summonerName']
                
                participants_row = {}
                
                #==== from match_details, i am using teams and inside teams, the amount of dragons killed====#
                for temp in match_detail['teams']:
                    if(r_name['player']['summonerName'] == "Desuuuuuuuuuu"):
                        participants_row['dragon kills'] = temp['dragonKills']
                 #==== from match_details, i am using teams and inside teams, the amount of dragons killed====#
                        
                #==== from match_details, i am using gameDuration ====#
                g = match_detail['gameDuration']/60
                participants_row['game duration'] = "{:.2f}".format(g)
                #==== from match_details, i am using gameDuration ====#
                
                #==== getting info from match_detail['stats'] ====#
                participants_row['turret kills'] = row['stats']['turretKills']
                participants_row['inhibitor kills'] = row['stats']['inhibitorKills']
                participants_row['gold earned'] = row['stats']['goldEarned']
                participants_row['first blood'] = row['stats']['firstBloodKill']
                participants_row['champion'] = row['championId']
                participants_row['kills'] = row['stats']['kills']
                participants_row['win'] = row['stats']['win']
                participants_row['deaths'] = row['stats']['deaths']
                participants_row['kill assists'] = row['stats']['assists']
                participants_row['total Damage Dealt'] = row['stats']['totalDamageDealt']
                participants_row['total Minions Killed'] = row['stats']['totalMinionsKilled']
                #==== getting info from match_detail['stats'] ====#
                
                participants.append(participants_row)
                participantsID.append(p_name)
                
                #==== Champions come in id form so this helps to convert it to string ====# 
                champ_dict = {}
                for key in static_champ_list['data']:
                    row = static_champ_list['data'][key]
                    champ_dict[row['key']] = row['id']
                for row in participants:
                    row['champion'] = champ_dict[str(row['champion'])]    
                #==== Champions come in id form so this helps to convert it to string ====# 
                
                df = pd.DataFrame(participants)
                dg = pd.DataFrame(participantsID)
                
                df_conc = pd.concat([dg,df], axis = 1, sort = False)
                return df_conc

In [9]:
#==== VIEW MY DATASET BEFORE EXPORT ====#

y = pd.DataFrame()
for i in range(10):
    z = pd.DataFrame(data = matchHistory(i))
    y = pd.concat([z,y])
y.reset_index(drop = True, inplace = True)
y

,name,champion,deaths,dragon kills,first blood,game duration,gold earned,inhibitor kills,kill assists,kills,total Damage Dealt,total Minions Killed,turret kills,win
0,Desuuuuuuuuuu,Shaco,2,1,False,11.80,5201,0,1,2,14541,29,0,False
1,Desuuuuuuuuuu,Aatrox,7,0,False,24.20,14146,1,16,6,120293,94,2,False
2,Desuuuuuuuuuu,Sona,8,0,False,16.37,6297,0,2,2,25997,42,0,False
3,Desuuuuuuuuuu,Singed,4,2,False,24.78,14992,1,15,9,104656,116,3,True
4,Desuuuuuuuuuu,Hecarim,5,1,False,24.53,17305,0,14,13,133680,97,3,True
5,Desuuuuuuuuuu,Leblanc,5,1,False,26.03,9923,0,3,3,56030,64,0,False
6,Desuuuuuuuuuu,Fiddlesticks,1,0,False,14.77,10890,0,5,9,61129,55,1,True
7,Desuuuuuuuuuu,Lux,19,3,False,29.03,12077,0,8,9,48727,29,0,False
8,Desuuuuuuuuuu,Talon,4,0,False,16.58,13193,0,7,18,51723,34,0,True
9,Desuuuuuuuuuu,Nunu,6,1,False,35.00,20740,0,13,14,115776,74,1,False


In [11]:
p = pd.DataFrame()
for i in range(100):   
    d = pd.DataFrame(data = matchHistory(i))
    p = pd.concat([d,p])
p.reset_index(drop = True, inplace = True)
p.to_csv('league.csv', index = False)

# Training dataset


In [64]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
data = pd.read_csv('league.csv')

## Using Kills and Kill assists to determine accuracy

In [60]:
X = data[['kills', 'kill assists']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.62      0.50      0.56        20
        True       0.41      0.54      0.47        13

    accuracy                           0.52        33
   macro avg       0.52      0.52      0.51        33
weighted avg       0.54      0.52      0.52        33



## Using Kills, Kill assists and Damage Dealth to determine accuracy

In [71]:
X = data[['kills', 'kill assists', 'total Damage Dealt']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.67      0.20      0.31        20
        True       0.41      0.85      0.55        13

    accuracy                           0.45        33
   macro avg       0.54      0.52      0.43        33
weighted avg       0.56      0.45      0.40        33



## Using Kill assists and Deaths to determine accuracy

In [61]:
X = data[['kill assists', 'deaths']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.85      0.85      0.85        20
        True       0.77      0.77      0.77        13

    accuracy                           0.82        33
   macro avg       0.81      0.81      0.81        33
weighted avg       0.82      0.82      0.82        33



## Using Kill assists, Deaths and total Damage Dealth to determine accuracy

In [72]:
X = data[['kill assists', 'deaths', 'total Damage Dealt']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.80      0.80      0.80        20
        True       0.69      0.69      0.69        13

    accuracy                           0.76        33
   macro avg       0.75      0.75      0.75        33
weighted avg       0.76      0.76      0.76        33



## Using Kills an Deaths to determine accuracy

In [62]:
X = data[['kills', 'deaths']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.78      0.70      0.74        20
        True       0.60      0.69      0.64        13

    accuracy                           0.70        33
   macro avg       0.69      0.70      0.69        33
weighted avg       0.71      0.70      0.70        33



## Using Kills, Deaths and total Damage Dealt to determine accuracy

In [73]:
X = data[['kills', 'deaths', 'total Damage Dealt']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.72      0.65      0.68        20
        True       0.53      0.62      0.57        13

    accuracy                           0.64        33
   macro avg       0.63      0.63      0.63        33
weighted avg       0.65      0.64      0.64        33



## Using Kills, Kill assists, Deaths to determine accuracy

In [48]:
X = data[['kills', 'kill assists', 'deaths']]
y = data['win']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=101)
logmodel = LogisticRegression(solver = 'lbfgs')
logmodel.fit(X_train, y_train)
predictions = logmodel.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

       False       0.82      0.70      0.76        20
        True       0.62      0.77      0.69        13

    accuracy                           0.73        33
   macro avg       0.72      0.73      0.72        33
weighted avg       0.75      0.73      0.73        33



## Using Kills, Kill assists, Deaths and total Damage to determine accuracy

In [56]:
X2 = data[['kills', 'kill assists', 'deaths','total Damage Dealt']]
y2 = data['win']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2, test_size=0.33, random_state=101)
logmodel2 = LogisticRegression(solver = 'lbfgs')
logmodel2.fit(X2_train, y2_train)
predictions2 = logmodel2.predict(X2_test)
print(classification_report(y2_test,predictions2))

              precision    recall  f1-score   support

       False       0.82      0.70      0.76        20
        True       0.62      0.77      0.69        13

    accuracy                           0.73        33
   macro avg       0.72      0.73      0.72        33
weighted avg       0.75      0.73      0.73        33



## Using gold earned and first blood to determine accuracy

In [84]:
X3 = data[[ 'gold earned', 'first blood']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.57      0.40      0.47        20
        True       0.37      0.54      0.44        13

    accuracy                           0.45        33
   macro avg       0.47      0.47      0.45        33
weighted avg       0.49      0.45      0.46        33



## Using first blood, dragon kills and game duration to determine accuracy

In [93]:
X3 = data[['first blood', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       1.00      0.35      0.52        20
        True       0.50      1.00      0.67        13

    accuracy                           0.61        33
   macro avg       0.75      0.68      0.59        33
weighted avg       0.80      0.61      0.58        33



## Using gold earned, dragon kills and game duration to determine accuracy

In [97]:
X3 = data[[ 'gold earned', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.88      0.35      0.50        20
        True       0.48      0.92      0.63        13

    accuracy                           0.58        33
   macro avg       0.68      0.64      0.57        33
weighted avg       0.72      0.58      0.55        33



##  Using gold earned, first blood, dragon kills and game Duration to determine accuracy

In [98]:
X3 = data[[ 'gold earned', 'first blood', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.88      0.35      0.50        20
        True       0.48      0.92      0.63        13

    accuracy                           0.58        33
   macro avg       0.68      0.64      0.57        33
weighted avg       0.72      0.58      0.55        33



##  Using Kills, gold earned, first blood, dragon kills and game Duration to determine accuracy

In [100]:
X3 = data[['kills', 'gold earned', 'first blood', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.86      0.60      0.71        20
        True       0.58      0.85      0.69        13

    accuracy                           0.70        33
   macro avg       0.72      0.72      0.70        33
weighted avg       0.75      0.70      0.70        33



##  Using Kill assists, gold earned, first blood, dragon kills and game Duration to determine accuracy

In [76]:
X3 = data[['kill assists' , 'gold earned', 'first blood', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.86      0.60      0.71        20
        True       0.58      0.85      0.69        13

    accuracy                           0.70        33
   macro avg       0.72      0.72      0.70        33
weighted avg       0.75      0.70      0.70        33



##  Using Deaths, gold earned, first blood, dragon kills and game Duration to determine accuracy

In [118]:
X3 = data[['deaths' , 'gold earned', 'first blood', 'dragon kills', 'game duration']]
y3 = data['win']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3,y3, test_size=0.33, random_state=101)
logmodel3 = LogisticRegression(solver = 'lbfgs')
logmodel3.fit(X3_train, y3_train)
predictions3 = logmodel3.predict(X3_test)
print(classification_report(y3_test,predictions3))

              precision    recall  f1-score   support

       False       0.79      0.55      0.65        20
        True       0.53      0.77      0.62        13

    accuracy                           0.64        33
   macro avg       0.66      0.66      0.64        33
weighted avg       0.68      0.64      0.64        33



## Using Inhib kills and Turret kills to determine accuracy

In [53]:
X4 = data[['inhibitor kills', 'turret kills']]
y4 = data['win']
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4, test_size=0.33, random_state=101)
logmodel4 = LogisticRegression(solver = 'lbfgs')
logmodel4.fit(X4_train, y4_train)
predictions4 = logmodel4.predict(X4_test)
print(classification_report(y4_test,predictions4))

              precision    recall  f1-score   support

       False       0.75      0.60      0.67        20
        True       0.53      0.69      0.60        13

    accuracy                           0.64        33
   macro avg       0.64      0.65      0.63        33
weighted avg       0.66      0.64      0.64        33



## Using Kills, Inhib kills and Turret kills to determine accuracy

In [80]:
X4 = data[['kills' , 'inhibitor kills', 'turret kills']]
y4 = data['win']
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4, test_size=0.33, random_state=101)
logmodel4 = LogisticRegression(solver = 'lbfgs')
logmodel4.fit(X4_train, y4_train)
predictions4 = logmodel4.predict(X4_test)
print(classification_report(y4_test,predictions4))

              precision    recall  f1-score   support

       False       0.75      0.60      0.67        20
        True       0.53      0.69      0.60        13

    accuracy                           0.64        33
   macro avg       0.64      0.65      0.63        33
weighted avg       0.66      0.64      0.64        33



## Using Kill assist, Inhib kills and Turret kills to determine accuracy

In [82]:
X4 = data[['kill assists', 'inhibitor kills', 'turret kills']]
y4 = data['win']
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4, test_size=0.33, random_state=101)
logmodel4 = LogisticRegression(solver = 'lbfgs')
logmodel4.fit(X4_train, y4_train)
predictions4 = logmodel4.predict(X4_test)
print(classification_report(y4_test,predictions4))

              precision    recall  f1-score   support

       False       0.73      0.55      0.63        20
        True       0.50      0.69      0.58        13

    accuracy                           0.61        33
   macro avg       0.62      0.62      0.60        33
weighted avg       0.64      0.61      0.61        33



## Using Deaths, Inhib kills and Turret kills to determine accuracy

In [83]:
X4 = data[['deaths', 'inhibitor kills', 'turret kills']]
y4 = data['win']
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4, test_size=0.33, random_state=101)
logmodel4 = LogisticRegression(solver = 'lbfgs')
logmodel4.fit(X4_train, y4_train)
predictions4 = logmodel4.predict(X4_test)
print(classification_report(y4_test,predictions4))

              precision    recall  f1-score   support

       False       0.75      0.75      0.75        20
        True       0.62      0.62      0.62        13

    accuracy                           0.70        33
   macro avg       0.68      0.68      0.68        33
weighted avg       0.70      0.70      0.70        33



## Using total Damage Dealth, gold earned, first blood, dragon kills, game Duration, Inhib kills and Turret kills to determine accuracy

In [79]:
X4 = data[['total Damage Dealt' , 'inhibitor kills', 'turret kills', 'gold earned', 'first blood', 'dragon kills', 'game duration']]
y4 = data['win']
X4_train, X4_test, y4_train, y4_test = train_test_split(X4,y4, test_size=0.33, random_state=101)
logmodel4 = LogisticRegression(solver = 'lbfgs')
logmodel4.fit(X4_train, y4_train)
predictions4 = logmodel4.predict(X4_test)
print(classification_report(y4_test,predictions4))

              precision    recall  f1-score   support

       False       0.86      0.60      0.71        20
        True       0.58      0.85      0.69        13

    accuracy                           0.70        33
   macro avg       0.72      0.72      0.70        33
weighted avg       0.75      0.70      0.70        33



## Using everything except for Kills and Kill assist

In [54]:
X5 = data[['inhibitor kills', 'deaths' , 'turret kills', 'total Damage Dealt','gold earned', 'first blood', 'dragon kills', 'game duration' ]]
y5 = data['win']
X5_train, X5_test, y5_train, y5_test = train_test_split(X5,y5, test_size=0.33, random_state=101)
logmodel5 = LogisticRegression(solver = 'lbfgs')
logmodel5.fit(X5_train, y5_train)
predictions5 = logmodel5.predict(X5_test)
print(classification_report(y5_test,predictions5))

              precision    recall  f1-score   support

       False       0.74      0.70      0.72        20
        True       0.57      0.62      0.59        13

    accuracy                           0.67        33
   macro avg       0.65      0.66      0.66        33
weighted avg       0.67      0.67      0.67        33



# Using everything except for Kills and Deaths

In [43]:
X6 = data[['inhibitor kills', 'kill assists' , 'turret kills', 'total Damage Dealt','gold earned', 'first blood', 'dragon kills', 'game duration' ]]
y6 = data['win']
X6_train, X6_test, y6_train, y6_test = train_test_split(X6,y6, test_size=0.33, random_state=101)
logmodel6 = LogisticRegression(solver = 'lbfgs')
logmodel6.fit(X6_train, y6_train)
predictions6 = logmodel6.predict(X6_test)
print(classification_report(y6_test,predictions6))

              precision    recall  f1-score   support

       False       0.86      0.60      0.71        20
        True       0.58      0.85      0.69        13

    accuracy                           0.70        33
   macro avg       0.72      0.72      0.70        33
weighted avg       0.75      0.70      0.70        33



# Using everything except for Deaths and Kill assist

In [66]:
X7 = data[['inhibitor kills', 'kills' , 'turret kills', 'total Damage Dealt','gold earned', 'first blood', 'dragon kills', 'game duration' ]]
y7 = data['win']
X7_train, X7_test, y7_train, y7_test = train_test_split(X7,y7, test_size=0.33, random_state=101)
logmodel7 = LogisticRegression(solver = 'lbfgs')
logmodel7.fit(X7_train, y7_train)
predictions7 = logmodel7.predict(X7_test)
print(classification_report(y7_test,predictions7))

              precision    recall  f1-score   support

       False       0.65      0.55      0.59        20
        True       0.44      0.54      0.48        13

    accuracy                           0.55        33
   macro avg       0.54      0.54      0.54        33
weighted avg       0.56      0.55      0.55        33



## Using everything

In [68]:
X8 = data[['inhibitor kills', 'kills', 'kill assists', 'deaths' , 'turret kills', 'total Damage Dealt','gold earned', 'first blood', 'dragon kills', 'game duration' ]]
y8 = data['win']
X8_train, X8_test, y8_train, y8_test = train_test_split(X8,y8, test_size=0.33, random_state=101)
logmodel8 = LogisticRegression(solver = 'lbfgs')
logmodel8.fit(X8_train, y8_train)
predictions8 = logmodel8.predict(X8_test)
print(classification_report(y8_test,predictions8))

              precision    recall  f1-score   support

       False       0.78      0.70      0.74        20
        True       0.60      0.69      0.64        13

    accuracy                           0.70        33
   macro avg       0.69      0.70      0.69        33
weighted avg       0.71      0.70      0.70        33



## Looking for highest possible accuracy

In [120]:
X8 = data[['kill assists', 'deaths', 'game duration', 'turret kills', 'inhibitor kills','dragon kills' ]]
y8 = data['win']
X8_train, X8_test, y8_train, y8_test = train_test_split(X8,y8, test_size=0.33, random_state=101)
logmodel8 = LogisticRegression(solver = 'lbfgs')
logmodel8.fit(X8_train, y8_train)
predictions8 = logmodel8.predict(X8_test)
print(classification_report(y8_test,predictions8))

              precision    recall  f1-score   support

       False       0.94      0.85      0.89        20
        True       0.80      0.92      0.86        13

    accuracy                           0.88        33
   macro avg       0.87      0.89      0.88        33
weighted avg       0.89      0.88      0.88        33



| Kills | Kill Assists | Deaths | Damage dealt | gold earned | first blood | dragon kills | game duration | turret kills | Inhib kills | Lose | Win |Accuracy |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ✓ | ✓ |  |  |  |  |  |  |  |  | 0.62 | 0.41 | 0.52 |
| ✓ | ✓ |  | ✓ |  |  |  |  |  |  | 0.67 | 0.41 | 0.45 |
|  | ✓ | ✓ |  | |  |  |  |  |  | 0.85 | 0.77 | 0.82 |
|  | ✓ | ✓ | ✓ | |  |  |  |  |  | 0.80 | 0.69 | 0.76 |
| ✓ |  | ✓ |  |  |  |  |  |  |  | 0.78 | 0.60 | 0.70 |
| ✓ |  | ✓ | ✓ |  |  |  |  |  |  | 0.72 | 0.53 | 0.64 |
| ✓ | ✓ | ✓ |  |  |  |  |  |  |  | 0.82 | 0.62 | 0.73 |
| ✓ | ✓ | ✓ | ✓ |  |  | |  |  |  | 0.82 | 0.62 | 0.73 |
|  |  |  |  | ✓ | ✓ |  |  |  |  | 0.57 | 0.37 | 0.45 |
|  |  |  |  |  | ✓ | ✓ | ✓ |  |  | 1.00 | 0.50 | 0.61 |
|  |  |  |  | ✓ |  | ✓ | ✓ |  |  | 0.88 | 0.48 | 0.58 |
|  |  |  |  | ✓ | ✓ | ✓ | ✓ |  |  | 0.88 | 0.48 | 0.58 |
| ✓ |  |  |  | ✓ | ✓ | ✓ | ✓ |  |  | 0.86 | 0.58 | 0.70 |
|  | ✓ |  |  | ✓ | ✓ | ✓ | ✓ |  |  | 0.86 | 0.58 | 0.70 |
|  |  | ✓ |  | ✓ | ✓ | ✓ | ✓ |  |  | 0.79 | 0.53 | 0.64 |
|  |  |  |  |  |  |  |  | ✓ | ✓ | 0.75 | 0.53 | 0.64 |
| ✓ |  |  |  |  |  |  |  | ✓ | ✓ | 0.75 | 0.53 | 0.64 |
|  | ✓ |  |  |  |  |  |  | ✓ | ✓ | 0.73 | 0.50 | 0.61 |
|  |  | ✓ |  |  |  |  |  | ✓ | ✓ | 0.75 | 0.62 | 0.70 |
|  |  |  | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | 0.86 | 0.58 | 0.70 |
|  |  | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | 0.74 | 0.57 | 0.67 |
|  | ✓ |  | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | 0.86 | 0.58 | 0.70 |
| ✓ |  |  | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | 0.65 | 0.44 | 0.55 |
| ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | 0.78 | 0.60 | 0.70 |
|  | ✓ | ✓ |  |  |  | ✓ | ✓ | ✓ | ✓ | 0.94 | 0.80 | 0.88 |